In [1]:
#remove cell visibility
from IPython.display import HTML
tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.input').hide()
    } else {
        $('div.input').show()
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
Toggle cell visibility <a href="javascript:code_toggle()">here</a>.''')
display(tag)

In [2]:
%matplotlib inline
import control
import numpy
import sympy as sym
from IPython.display import display, Markdown
import ipywidgets as widgets
import matplotlib.pyplot as plt


#print a matrix latex-like
def bmatrix(a):
     """Returns a LaTeX bmatrix - by Damir Arbula (ICCT project)

     :a: numpy array
     :returns: LaTeX bmatrix as a string
     """
     if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
     lines = str(a).replace('[', '').replace(']', '').splitlines()
     rv = [r'\begin{bmatrix}']
     rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
     rv +=  [r'\end{bmatrix}']
     return '\n'.join(rv)


# Display formatted matrix: 
def vmatrix(a):
    if len(a.shape) > 2:
         raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{vmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{vmatrix}']
    return '\n'.join(rv)


#matrixWidget is a matrix looking widget built with a VBox of HBox(es) that returns a numPy array as value !
class matrixWidget(widgets.VBox):
    def updateM(self,change):
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.M_[irow,icol] = self.children[irow].children[icol].value
                #print(self.M_[irow,icol])
        self.value = self.M_

    def dummychangecallback(self,change):
        pass
    
    
    def __init__(self,n,m):
        self.n = n
        self.m = m
        self.M_ = numpy.matrix(numpy.zeros((self.n,self.m)))
        self.value = self.M_
        widgets.VBox.__init__(self,
                             children = [
                                 widgets.HBox(children = 
                                              [widgets.FloatText(value=0.0, layout=widgets.Layout(width='90px')) for i in range(m)]
                                             ) 
                                 for j in range(n)
                             ])
        
        #fill in widgets and tell interact to call updateM each time a children changes value
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
                self.children[irow].children[icol].observe(self.updateM, names='value')
        #value = Unicode('example@example.com', help="The email value.").tag(sync=True)
        self.observe(self.updateM, names='value', type= 'All')
        
    def setM(self, newM):
        #disable callbacks, change values, and reenable
        self.unobserve(self.updateM, names='value', type= 'All')
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].unobserve(self.updateM, names='value')
        self.M_ = newM
        self.value = self.M_
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].value = self.M_[irow,icol]
        for irow in range(0,self.n):
            for icol in range(0,self.m):
                self.children[irow].children[icol].observe(self.updateM, names='value')
        self.observe(self.updateM, names='value', type= 'All')        

                #self.children[irow].children[icol].observe(self.updateM, names='value')

             
#overlaod class for state space systems that DO NOT remove "useless" states (what "professor" of automatic control would do this?)
class sss(control.StateSpace):
    def __init__(self,*args):
        #call base class init constructor
        control.StateSpace.__init__(self,*args)
    #disable function below in base class
    def _remove_useless_states(self):
        pass

## Rugalmas csuklós robotkar szabályozása

A robotkart egy elektromos motor hajtja egy rugalmas csuklón át, ami torziós rugóként viselkedik. A rendszer egy háromtárolós lineáris időinvariáns modellel közelíthető, ahol az állapotok:
- $x_1$ : a különbsége a motor és a kar szögeinek (nem nulla a rugalmasság miatt),
- $x_2$ : a motor tengelyének szögsebessége,
- $x_3$ : a kar szögsebessége.
A bemenet $u$ a motor nyomatéka. A rendszer egyenletei:

\begin{cases}
\dot{x} = \begin{bmatrix} 0 & 1 & -1 \\ a-1 & -b_1 & b_1 \\ a & b_2 & -b_2 \end{bmatrix}x + \begin{bmatrix} 0 \\ b_3 \\ 0 \end{bmatrix}u \\
y = \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}x
\end{cases}

ahol $a=0.1$, $b_1=0.09$, $b_2=0.01$ és $b_3=90$.

A szabályozó tervezés célja egy olyan regulátor létrehozása ami a kar szögsebességét úgy befolyásolja, hogy a domináns pólus csillapítása 0.7, sajátkörfrekvenciája pedig 0.5 rad/s legyen, a maradó hiba pedig nulla legyen sebesség ugrás hatására.

A rendszer átviteli függvénye:

In [3]:
A = numpy.matrix('0 1 -1; -0.9 -0.09 0.09; 0.1 0.01 -0.01')
B = numpy.matrix('0; 90; 0')
C = numpy.matrix('0 0 1')
D = numpy.matrix('0')
sys_tf = control.tf(sss(A,B,C,D))
print(sys_tf)


  3.886e-16 s^2 + 0.9 s + 9
-----------------------------
s^3 + 0.1 s^2 + s - 8.674e-19



pólusai

In [4]:
import warnings
# In order to suppress the warning BadCoefficient
warnings.filterwarnings("ignore")
print(numpy.round(sys_tf.pole(),3))

[-0.05+0.999j -0.05-0.999j  0.  +0.j   ]


és zérusai

In [5]:
print(numpy.round(sys_tf.zero(),3),'.')

[-2.31613695e+15 -1.00000000e+01] .


Vizsgáljuk a rendszert, hogy eldöntsük, irányítható, illetve megfigyelhető-e. Az irányíthatósági mátrix $\mathcal{C}$

In [6]:
Ctrb = control.ctrb(A,B)
display(Markdown(bmatrix(Ctrb)))
# print(numpy.linalg.matrix_rank(Ctrb))

\begin{bmatrix}
  0. & 90. & -9.\\
  90. & -8.1 & -80.19\\
  0. & 0.9 & 8.91\\
\end{bmatrix}

és rangjának 3-nak kell lennie az irányíthatósághoz. A megfigyelhetőségi mátrix $\mathcal{O}$

In [7]:
Obsv = control.obsv(A,C)
display(Markdown(bmatrix(Obsv)))
# print(numpy.linalg.matrix_rank(Obsv))

\begin{bmatrix}
  0. & 0. & 1.\\
  0.1 & 0.01 & -0.01\\
  -0.01 & 0.099 & -0.099\\
\end{bmatrix}

és rangjának 3-nak kell lennie a megfigyelhetőséghez.

Ezt annak alapján is el lehet dönteni, hogy az átviteli függvény nevezője harmadrendű (egyezik az állapotvektor elemeinek számával).

### Regulátor tervezése
#### Szabályozó tervezése

A követelmények teljesítéséhez két pólust kell elhelyeznünk $\zeta \omega_n \pm \sqrt{1-\zeta^2}\omega_n = -0.35\pm0.357i$-ben, a fennmaradó valós pólust pedig egy, a komplex (domináns) póluspárnál nagyobb frekvenciára. Így rendszer harmadik pólusát -3.5 rad/s-be helyezzük. Hogy teljesüljön a nulla maradó hiba, a referenciát megszorozzuk a zárt kör statikus erősítésének inverzével.

#### Megfigyelő tervezése

Ahhoz, hogy a megfigyelő megfelelő sebességgel működjön, a pólusait -10 rad/s-be helyezzük.

### Hogyan működik a példa?
- Ellenőrizze, hogy a zárt hurkú rendszer jól teljesít-e kezdeti becslési hiba esetéb. Próbálja javítani a teljesítményét.
- Csökkentse a frekvenciáját a valós pólusának a szabályozott zárt huroknak és figyelje meg hogyan változik a rendszer válasza.

In [8]:
# Preparatory cell

X0 = numpy.matrix('0.0; 0.0; 0.0')
K = numpy.matrix([8/15,-4.4,-4])
L = numpy.matrix([[23],[66],[107/3]])

Aw = matrixWidget(3,3)
Aw.setM(A)
Bw = matrixWidget(3,1)
Bw.setM(B)
Cw = matrixWidget(1,3)
Cw.setM(C)
X0w = matrixWidget(3,1)
X0w.setM(X0)
Kw = matrixWidget(1,3)
Kw.setM(K)
Lw = matrixWidget(3,1)
Lw.setM(L)


eig1c = matrixWidget(1,1)
eig2c = matrixWidget(2,1)
eig3c = matrixWidget(1,1)
eig1c.setM(numpy.matrix([-3.5])) 
eig2c.setM(numpy.matrix([[-0.35],[-0.357]]))
eig3c.setM(numpy.matrix([-3.5]))

eig1o = matrixWidget(1,1)
eig2o = matrixWidget(2,1)
eig3o = matrixWidget(1,1)
eig1o.setM(numpy.matrix([-10.])) 
eig2o.setM(numpy.matrix([[-10.],[0.]]))
eig3o.setM(numpy.matrix([-10.]))

In [9]:
# Misc

#create dummy widget 
DW = widgets.FloatText(layout=widgets.Layout(width='0px', height='0px'))

#create button widget
START = widgets.Button(
    description='Vizsgálat',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Vizsgálat',
    icon='check'
)
                       
def on_start_button_clicked(b):
    #This is a workaround to have intreactive_output call the callback:
    #   force the value of the dummy widget to change
    if DW.value> 0 :
        DW.value = -1
    else: 
        DW.value = 1
    pass
START.on_click(on_start_button_clicked)

# Define type of method 
selm = widgets.Dropdown(
    options= ['K és L megadása', 'Sajátértékek megadása'],
    value= 'Sajátértékek megadása',
    description='',
    disabled=False
)

# Define the number of complex eigenvalues
sele = widgets.Dropdown(
    options= ['0 komplex sajátérték', '2 komplex sajátérték'],
    value= '2 komplex sajátérték',
    description='Komplex sajátértékek:',
    style = {'description_width': 'initial'},
    disabled=False
)

#define type of ipout 
selu = widgets.Dropdown(
    options=['impulzus', 'egységugrás', 'szinusz', 'négyszögjel'],
    value='egységugrás',
    description='Referencia típusa:',
    style = {'description_width': 'initial'},
    disabled=False
)
# Define the values of the input
u = widgets.FloatSlider(
    value=1,
    min=0,
    max=3,
    step=0.1,
    description='Referencia:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
period = widgets.FloatSlider(
    value=0.5,
    min=0.001,
    max=10,
    step=0.001,
    description='Periódus: ',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

gain_w2 = widgets.FloatText(
    value=1.,
    description='',
    disabled=True
)

simTime = widgets.FloatText(
    value=20,
    description='',
    disabled=False
)

In [10]:
# Support functions

def eigen_choice(sele):
    if sele == '0 komplex sajátérték':
        eig1c.children[0].children[0].disabled = False
        eig2c.children[1].children[0].disabled = True
        eig1o.children[0].children[0].disabled = False
        eig2o.children[1].children[0].disabled = True
        eig = 0
    if sele == '2 komplex sajátérték':
        eig1c.children[0].children[0].disabled = True
        eig2c.children[1].children[0].disabled = False
        eig1o.children[0].children[0].disabled = True
        eig2o.children[1].children[0].disabled = False
        eig = 2
    return eig

def method_choice(selm):
    if selm == 'K és L megadása':
        method = 1
        sele.disabled = True
    if selm == 'Sajátértékek megadása':
        method = 2
        sele.disabled = False
    return method

In [11]:
s = control.tf('s')
Gref = (0.5)**2/(s**2 + 2*0.7*0.5*s + (0.5)**2)

def main_callback2(Aw, Bw, X0w, K, L, eig1c, eig2c, eig3c, eig1o, eig2o, eig3o, u, period, selm, sele, selu, simTime, DW):
    eige = eigen_choice(sele)
    method = method_choice(selm)
    
    if method == 1:
        solc = numpy.linalg.eig(A-B*K)
        solo = numpy.linalg.eig(A-L*C)
    if method == 2:
        if eige == 0:
            K = control.acker(A, B, [eig1c[0,0], eig2c[0,0], eig3c[0,0]])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig1o[0,0], eig2o[0,0], eig3o[0,0]]).T
            Lw.setM(L)
        if eige == 2:
            K = control.acker(A, B, [eig3c[0,0], 
                                     numpy.complex(eig2c[0,0],eig2c[1,0]), 
                                     numpy.complex(eig2c[0,0],-eig2c[1,0])])
            Kw.setM(K)
            L = control.acker(A.T, C.T, [eig3o[0,0], 
                                         numpy.complex(eig2o[0,0],eig2o[1,0]), 
                                         numpy.complex(eig2o[0,0],-eig2o[1,0])]).T
            Lw.setM(L)
            
    
    sys = control.ss(A,B,numpy.vstack((C,numpy.zeros((B.shape[1],C.shape[1])))),numpy.vstack((D,numpy.eye(B.shape[1]))))
    sysC = control.ss(numpy.zeros((1,1)),
                      numpy.zeros((1,numpy.shape(A)[0])),
                      numpy.zeros((numpy.shape(B)[1],1)),
                      -K)
    
    sysE = control.ss(A-L*C,
                      numpy.hstack((L,B-L*D)),
                      numpy.eye(numpy.shape(A)[0]),
                      numpy.zeros((A.shape[0],C.shape[0]+B.shape[1])))
    
    sys_append = control.append(sys, sysE, sysC, control.ss(A,B,numpy.eye(A.shape[0]),numpy.zeros((A.shape[0],B.shape[1]))))
    Q = []
    # y in ingresso a sysE
    for i in range(C.shape[0]):
        Q.append([B.shape[1]+i+1, i+1])
    # u in ingresso a sysE
    for i in range(B.shape[1]):
        Q.append([B.shape[1]+C.shape[0]+i+1, C.shape[0]+i+1])
    # u in ingresso a sys
    for i in range(B.shape[1]):
        Q.append([i+1, C.shape[0]+B.shape[1]+A.shape[0]+i+1])
    # u in ingresso al sistema che ha come uscite gli stati reali
    for i in range(B.shape[1]):
        Q.append([2*B.shape[1]+C.shape[0]+A.shape[0]+i+1, C.shape[0]+i+1])
    # xe in ingresso a sysC
    for i in range(A.shape[0]):
        Q.append([2*B.shape[1]+C.shape[0]+i+1, C.shape[0]+B.shape[1]+i+1])
        
    inputv = [i+1 for i in range(B.shape[1])]
    outputv = [i+1 for i in range(numpy.shape(sys_append.C)[0])]
    sys_CL = control.connect(sys_append,
                             Q,
                             inputv,
                             outputv)
    
    t = numpy.linspace(0, 100000, 2)
    t, yout = control.step_response(sys_CL[0,0],T=t)
    dcgain = yout[-1]
    gain_w2.value = dcgain
    if dcgain != 0:
        u1 = u/gain_w2.value
    else:
        print('A beállított előrecsatolási erősítés 0, így 1-re lett átállítva')
        u1 = u/1
    print('A zárt kör statikus erősítése (a referenciától a kimenetre): %.5f' %dcgain)
    
    X0w1 = numpy.zeros((A.shape[0],1))
    for j in range(A.shape[0]):
        X0w1 = numpy.vstack((X0w1,X0w[j]))
    X0w1 = numpy.vstack((X0w1,numpy.zeros((A.shape[0],1))))
    if simTime != 0:
        T = numpy.linspace(0, simTime, 10000)
    else:
        T = numpy.linspace(0, 1, 10000)
      
    if selu == 'impulzus': #selu
        U = [0 for t in range(0,len(T))]
        U[0] = u
        U1 = [0 for t in range(0,len(T))]
        U1[0] = u1
        T, yout, xout = control.forced_response(sys_CL,T,U1,X0w1)
        T, yout_ref, xout_ref = control.forced_response(Gref,T,U,[0, 0])
    if selu == 'egységugrás':
        U = [u for t in range(0,len(T))]
        U1 = [u1 for t in range(0,len(T))]
        T, yout, xout = control.forced_response(sys_CL,T,U1,X0w1)
        T, yout_ref, xout_ref = control.forced_response(Gref,T,U,[0, 0])
    if selu == 'szinusz':
        U = u*numpy.sin(2*numpy.pi/period*T)
        U1 = u1*numpy.sin(2*numpy.pi/period*T)
        T, yout, xout = control.forced_response(sys_CL,T,U1,X0w1)
        T, yout_ref, xout_ref = control.forced_response(Gref,T,U,[0, 0])
    if selu == 'négyszögjel':
        U = u*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        U1 = u1*numpy.sign(numpy.sin(2*numpy.pi/period*T))
        T, yout, xout = control.forced_response(sys_CL,T,U1,X0w1)
        T, yout_ref, xout_ref = control.forced_response(Gref,T,U,[0, 0])
    # N.B. i primi 3 stati di xout sono quelli del sistema, mentre gli ultimi 3 sono quelli dell'osservatore
    
    step_info_dict = control.step_info(sys_CL[0,0],SettlingTimeThreshold=0.05,T=T)
    print('Egységugrás: \n\tFelfutási idő =',step_info_dict['RiseTime'],'\n\tBeállási idő (5%) =',step_info_dict['SettlingTime'],'\n\tTúllövés (%)=',step_info_dict['Overshoot'])
    # print('Max x3 value (%)=', max(abs(yout[C.shape[0]+2*B.shape[1]+A.shape[0]+2]))/(numpy.pi/180*17)*100)
    
    fig = plt.figure(num='Szimuláció', figsize=(14,12))
    
    fig.add_subplot(221)
    plt.title('Kimenet válasza')
    plt.ylabel('Kimenet')
    plt.plot(T,yout[0],T,yout_ref,T,U,'r--')
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$y$','Másodrendű rendszer referencia','Referencia'])
    plt.grid()
    
    fig.add_subplot(222)
    plt.title('Bemenet')
    plt.ylabel('$u$')
    plt.plot(T,yout[C.shape[0]])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.grid()
    
    fig.add_subplot(223)
    plt.title('Állapotok válasza')
    plt.ylabel('Állapotok')
    plt.plot(T,yout[C.shape[0]+2*B.shape[1]+A.shape[0]],
             T,yout[C.shape[0]+2*B.shape[1]+A.shape[0]+1],
             T,yout[C.shape[0]+2*B.shape[1]+A.shape[0]+2])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$x_{1}$','$x_{2}$','$x_{3}$'])
    plt.grid()
    
    fig.add_subplot(224)
    plt.title('Becslési hiba')
    plt.ylabel('Hiba')
    plt.plot(T,yout[C.shape[0]+2*B.shape[1]+A.shape[0]]-yout[C.shape[0]+B.shape[1]],
             T,yout[C.shape[0]+2*B.shape[1]+A.shape[0]+1]-yout[C.shape[0]+B.shape[1]+1],
             T,yout[C.shape[0]+2*B.shape[1]+A.shape[0]+2]-yout[C.shape[0]+B.shape[1]+2])
    plt.xlabel('$t$ [s]')
    plt.axvline(x=0,color='black',linewidth=0.8)
    plt.axhline(y=0,color='black',linewidth=0.8)
    plt.legend(['$e_{1}$','$e_{2}$','$e_{3}$'])
    plt.grid()
    #plt.tight_layout()
   
alltogether2 = widgets.VBox([widgets.HBox([selm, 
                                          sele,
                                          selu]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('K:',border=3), Kw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Sajátértékek:',border=3), 
                                          eig1c, 
                                          eig2c, 
                                          eig3c,
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('X0 becs.:',border=3), X0w]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([widgets.Label('L:',border=3), Lw, 
                                          widgets.Label(' ',border=3),
                                          widgets.Label(' ',border=3),
                                          widgets.Label('Sajátértékek:',border=3), 
                                          eig1o, 
                                          eig2o, 
                                          eig3o,
                                          widgets.Label(' ',border=3),
                                          widgets.VBox([widgets.Label('Inverz referencia erősítés:',border=3),
                                                        widgets.Label('Szimulációs idő [s]:',border=3)]),
                                          widgets.VBox([gain_w2,simTime])]),
                            widgets.Label(' ',border=3),
                            widgets.HBox([u, 
                                          period, 
                                          START])])
out2 = widgets.interactive_output(main_callback2, {'Aw':Aw, 'Bw':Bw, 'X0w':X0w, 'K':Kw, 'L':Lw,
                                                 'eig1c':eig1c, 'eig2c':eig2c, 'eig3c':eig3c, 'eig1o':eig1o, 'eig2o':eig2o, 'eig3o':eig3o, 
                                                 'u':u, 'period':period, 'selm':selm, 'sele':sele, 'selu':selu, 'simTime':simTime, 'DW':DW})
out2.layout.height = '860px'
display(out2, alltogether2)

Output(layout=Layout(height='860px'))